In [ ]:
%load_ext autoreload

In [ ]:
%autoreload
import pandas as pd
import polars as pl
from utils import (
    agg_tnc,
    df_to_excel_col_dict,
    load_hh_raw,
    load_person_assign_day,
    load_tnc_trips,
    out_dir,
    tour_extract_wkday_dir,
    trip_join_hh_person,
    write_to_excel,
)
from xlsxwriter.utility import xl_rowcol_to_cell


In [ ]:
out_filepath = out_dir / "09_TNCTrips_WaitTime.xlsx"

In [ ]:
trip = trip_join_hh_person(
    pl.from_pandas(load_tnc_trips(tour_extract_wkday_dir, tnc_wait_time=True)).filter(
        pl.col("tnc_wait_time") < 100
    ),
    load_hh_raw(income=True, home_county=False),
    # TODO do we have to load person-assign_day, or can we just load 2a person-reformat?
    load_person_assign_day(  # TODO add back race/ethnicity
        tour_extract_wkday_dir / "person-assign_day.csv", demographics=True
    ),
).to_pandas()

In [ ]:
writer = pd.ExcelWriter(out_filepath, engine="xlsxwriter")
workbook = writer.book
format1 = workbook.add_format({"num_format": "#,##0.0"})

In [ ]:
PREF = "Weekday Avg TNC Wait Time by "
for key in ["raceeth", "hh_income_broad_cat", "pagey", "pgend", "tnc_type"]:
    d1_dict = df_to_excel_col_dict[key]
    row = 0
    sname = d1_dict["desc"]
    title = PREF + d1_dict["desc"]
    tab = agg_tnc(
        trip, d1_dict["col"], d1_dict["vals"], d1_dict["labels"], "tnc_wait_time"
    )
    row = write_to_excel(writer, tab, sname, title, row)
    tab_range = xl_rowcol_to_cell(row, 1) + ":" + xl_rowcol_to_cell(row, tab.shape[1])
    writer.sheets[sname].set_column(tab_range, 18, format1)

In [ ]:
writer.close()